# Target / mean encoding

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
# let's load the data set

data = pd.read_csv("credit_approval_uci.csv")

data.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,target
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,202.0,0,1
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,43.0,560,1
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,280.0,824,1
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,100.0,3,1
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,120.0,0,1


In [3]:
# Let's separate into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(
    data.drop(labels=["target"], axis=1),  # predictors
    data["target"],  # target
    test_size=0.3,  # percentage of observations in test set
    random_state=0,  # seed to ensure reproducibility
)

X_train.shape, X_test.shape

((483, 15), (207, 15))

# Target mean encoding with pandas

In [4]:
# let's create a dictionary with the target mean value
# per category for the variable A7

mapping = y_train.groupby(X_train["A7"]).mean().to_dict()

mapping

{'Missing': 1.0,
 'bb': 0.5128205128205128,
 'dd': 0.4,
 'ff': 0.14634146341463414,
 'h': 0.6039603960396039,
 'j': 0.2,
 'n': 0.6666666666666666,
 'o': 0.0,
 'v': 0.4187725631768953,
 'z': 0.7142857142857143}

In [5]:
# replace the labels with the target mean

X_train_enc = X_train.copy()
X_test_enc = X_test.copy()

X_train_enc["A7"] = X_train_enc["A7"].map(mapping)
X_test_enc["A7"] = X_test_enc["A7"].map(mapping)

# Target encoding with Scikit-learn

In [6]:
from sklearn.preprocessing import TargetEncoder
from sklearn.compose import ColumnTransformer

In [7]:
# let's select the categorical variables

cat_vars = X_train.select_dtypes(include="O").columns.to_list()

cat_vars

['A1', 'A4', 'A5', 'A6', 'A7', 'A9', 'A10', 'A12', 'A13']

In [8]:
enc = TargetEncoder(smooth="auto", random_state=9)

In [9]:
# Indicate which variables to encode:

ct = ColumnTransformer(
    [("encoder", enc, cat_vars)],
    remainder="passthrough",
).set_output(transform="pandas")

In [10]:
# let's transform train and test set

X_train_enc = ct.fit_transform(X_train, y_train)
X_test_enc = ct.transform(X_test)

In [11]:
ct.named_transformers_["encoder"].encodings_

[array([0.48977085, 0.47206282, 0.43883698]),
 array([1.        , 1.        , 0.51222159, 0.22745101]),
 array([1.        , 0.51222159, 1.        , 0.22745101]),
 array([1.        , 0.44140706, 0.45158801, 0.69310186, 0.19793494,
        0.61167676, 0.10867498, 0.27172849, 0.267243  , 0.2928946 ,
        0.4240346 , 0.62207703, 1.        , 0.49895426, 0.78241398]),
 array([1.        , 0.51121681, 0.4080061 , 0.15002659, 0.60249387,
        0.22854703, 0.61657956, 0.        , 0.41888051, 0.68635092]),
 array([0.07092592, 0.78426413]),
 array([0.25147697, 0.70182461]),
 array([0.45245936, 0.44547181]),
 array([0.46481717, 1.        , 0.21467342])]

In [12]:
X_train_enc.head()

,encoder__A1,encoder__A4,encoder__A5,encoder__A6,encoder__A7,encoder__A9,encoder__A10,encoder__A12,encoder__A13,remainder__A2,remainder__A3,remainder__A8,remainder__A11,remainder__A14,remainder__A15
596,0.475949,0.517129,0.517129,0.432675,0.408258,0.812012,0.719660,0.454524,0.467189,46.08,3.000,2.375,8,396.0,4159
303,0.474353,0.513565,0.513565,0.655568,0.430705,0.073196,0.250681,0.449070,0.460417,15.92,2.875,0.085,0,120.0,0
204,0.450182,0.235691,0.235691,0.486887,0.408253,0.759674,0.687389,0.458086,0.466809,36.33,2.125,0.085,1,50.0,1187
351,0.431885,0.277459,0.277459,0.100749,0.129423,0.067201,0.249536,0.456697,0.466055,22.17,0.585,0.000,0,100.0,0
118,0.450182,0.510204,0.510204,0.377833,0.408253,0.759674,0.687389,0.440268,0.466809,57.83,7.040,14.000,6,360.0,1332


# Target mean encoding with Feature-engine

In [13]:
from feature_engine.encoding import MeanEncoder

In [14]:
# let's create the encoder that automatically finds
# the categorical variables

mean_enc = MeanEncoder(smoothing="auto", variables=None)

In [15]:
# with fit, the transformer learns the numerical
# representation for each category, that is, the
# mean target value per category

# when fitting the transformer, we need to pass the target as well
# just like with any Scikit-learn predictor class

mean_enc.fit(X_train, y_train)

MeanEncoder(smoothing='auto')

In [16]:
# variables that will be encoded

mean_enc.variables_

['A1', 'A4', 'A5', 'A6', 'A7', 'A9', 'A10', 'A12', 'A13']

In [17]:
# in the encoder_dict_ we find the target mean per category
# per variable

mean_enc.encoder_dict_

{'A1': {'Missing': 0.4897708500995383,
  'a': 0.47206282385504167,
  'b': 0.4388369819348539},
 'A4': {'Missing': 1.0,
  'l': 1.0,
  'u': 0.5122215930819429,
  'y': 0.22745101171497045},
 'A5': {'Missing': 1.0,
  'g': 0.5122215930819429,
  'gg': 1.0,
  'p': 0.22745101171497045},
 'A6': {'Missing': 1.0,
  'aa': 0.4414070613807244,
  'c': 0.45158801273269944,
  'cc': 0.6931018626679666,
  'd': 0.19793494395099673,
  'e': 0.6116767599605843,
  'ff': 0.10867497814869279,
  'i': 0.2717284862549896,
  'j': 0.26724299829766096,
  'k': 0.2928945960903988,
  'm': 0.42403460050172886,
  'q': 0.622077031849362,
  'r': 1.0,
  'w': 0.4989542602527962,
  'x': 0.7824139821749067},
 'A7': {'Missing': 1.0,
  'bb': 0.5112168095503893,
  'dd': 0.4080060972835237,
  'ff': 0.15002659488133335,
  'h': 0.6024938698481956,
  'j': 0.22854703116167752,
  'n': 0.6165795586527293,
  'o': 0.0,
  'v': 0.41888050693409623,
  'z': 0.6863509157249638},
 'A9': {'f': 0.07092591902432215, 't': 0.7842641326398047},
 'A10'

In [18]:
# let's transform train and test set

X_train_enc = mean_enc.transform(X_train)
X_test_enc = mean_enc.transform(X_test)

In [19]:
# Let's inspect the transformed dataset

X_train_enc.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15
596,0.472063,46.08,3.000,0.512222,0.512222,0.451588,0.418881,2.375,0.784264,0.701825,8,0.445472,0.464817,396.0,4159
303,0.472063,15.92,2.875,0.512222,0.512222,0.622077,0.418881,0.085,0.070926,0.251477,0,0.452459,0.464817,120.0,0
204,0.438837,36.33,2.125,0.227451,0.227451,0.498954,0.418881,0.085,0.784264,0.701825,1,0.452459,0.464817,50.0,1187
351,0.438837,22.17,0.585,0.227451,0.227451,0.108675,0.150027,0.000,0.070926,0.251477,0,0.452459,0.464817,100.0,0
118,0.438837,57.83,7.040,0.512222,0.512222,0.424035,0.418881,14.000,0.784264,0.701825,6,0.445472,0.464817,360.0,1332


In [20]:
# Let's inspect the transformed dataset

X_test_enc.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15
14,0.472063,45.83,10.50,0.512222,0.512222,0.622077,0.418881,5.000,0.784264,0.701825,7,0.445472,0.464817,0.0,0
586,0.438837,64.08,20.00,0.512222,0.512222,0.782414,0.602494,17.500,0.784264,0.701825,9,0.445472,0.464817,0.0,1000
140,0.472063,31.25,3.75,0.512222,0.512222,0.693102,0.602494,0.625,0.784264,0.701825,9,0.445472,0.464817,181.0,0
492,0.438837,39.25,9.50,0.512222,0.512222,0.424035,0.418881,6.500,0.784264,0.701825,14,0.452459,0.464817,240.0,4607
350,0.472063,26.17,2.00,0.512222,0.512222,0.267243,0.228547,0.000,0.070926,0.251477,0,0.445472,0.464817,276.0,1


# Target mean encoding with Category Encoders

In [21]:
from category_encoders.target_encoder import TargetEncoder

In [22]:
# the parameter min_samples_leaf corresponds to the minimum
# number of observations per category group needed to
# estimate the target mean only from the posterior probability

# the posterior probability is the target mean as estimated in the
# previous cells using pandas

# for those categories with less than 25 observations, the target
# estimate will be a "blended" probability

mean_enc = TargetEncoder(cols=None, min_samples_leaf=25, smoothing=1.0)

In [23]:
# with fit, the transformer learns the numerical
# representation for each category

mean_enc.fit(X_train, y_train)

TargetEncoder(cols=['A1', 'A4', 'A5', 'A6', 'A7', 'A9', 'A10', 'A12', 'A13'],
              min_samples_leaf=25, smoothing=1.0)

In [24]:
# let's transform train and test set

X_train_enc = mean_enc.transform(X_train)
X_test_enc = mean_enc.transform(X_test)

In [25]:
# Let's inspect the transformed data

X_train_enc.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15
596,0.472222,46.08,3.000,0.512397,0.512397,0.451613,0.418773,2.375,0.785156,0.702830,8,0.445455,0.464853,396.0,4159
303,0.472222,15.92,2.875,0.512397,0.512397,0.625000,0.418773,0.085,0.070485,0.250923,0,0.452471,0.464853,120.0,0
204,0.438806,36.33,2.125,0.226087,0.226087,0.500000,0.418773,0.085,0.785156,0.702830,1,0.452471,0.464853,50.0,1187
351,0.438806,22.17,0.585,0.226087,0.226087,0.105264,0.146341,0.000,0.070485,0.250923,0,0.452471,0.464853,100.0,0
118,0.438806,57.83,7.040,0.512397,0.512397,0.430123,0.418773,14.000,0.785156,0.702830,6,0.445455,0.464853,360.0,1332


In [26]:
# Let's inspect the transformed data

X_test_enc.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15
14,0.472222,45.83,10.50,0.512397,0.512397,0.625000,0.418773,5.000,0.785156,0.702830,7,0.445455,0.464853,0.0,0
586,0.438806,64.08,20.00,0.512397,0.512397,0.541359,0.603960,17.500,0.785156,0.702830,9,0.445455,0.464853,0.0,1000
140,0.472222,31.25,3.75,0.512397,0.512397,0.698322,0.603960,0.625,0.785156,0.702830,9,0.445455,0.464853,181.0,0
492,0.438806,39.25,9.50,0.512397,0.512397,0.430123,0.418773,6.500,0.785156,0.702830,14,0.452471,0.464853,240.0,4607
350,0.472222,26.17,2.00,0.512397,0.512397,0.449275,0.449275,0.000,0.070485,0.250923,0,0.445455,0.464853,276.0,1


# M-estimate encoding with Category Encoders

In [27]:
from category_encoders.m_estimate import MEstimateEncoder

In [28]:
# Set up the encoder

# m determines the weight given to the prior, that is
# the target mean over the entire dataset

# bigger m values mean more weight to the prior

mean_enc = MEstimateEncoder(cols=None, m=1.0)

In [29]:
# with fit, the transformer learns the numerical
# representation for each category

mean_enc.fit(X_train, y_train)

MEstimateEncoder(cols=['A1', 'A4', 'A5', 'A6', 'A7', 'A9', 'A10', 'A12', 'A13'])

In [30]:
# let's transform train and test set

X_train_enc = mean_enc.transform(X_train)
X_test_enc = mean_enc.transform(X_test)

In [31]:
# Let's inspect the transformed data

X_train_enc.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15
596,0.472064,46.08,3.000,0.512223,0.512223,0.451588,0.418882,2.375,0.783849,0.701640,8,0.445472,0.464817,396.0,4159
303,0.472064,15.92,2.875,0.512223,0.512223,0.621917,0.418882,0.085,0.072146,0.251652,0,0.452459,0.464817,120.0,0
204,0.438837,36.33,2.125,0.228011,0.228011,0.498965,0.418882,0.085,0.783849,0.701640,1,0.452459,0.464817,50.0,1187
351,0.438837,22.17,0.585,0.228011,0.228011,0.114084,0.153554,0.000,0.072146,0.251652,0,0.452459,0.464817,100.0,0
118,0.438837,57.83,7.040,0.512223,0.512223,0.424047,0.418882,14.000,0.783849,0.701640,6,0.445472,0.464817,360.0,1332


In [32]:
# Let's inspect the transformed data

X_test_enc.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15
14,0.472064,45.83,10.50,0.512223,0.512223,0.621917,0.418882,5.000,0.783849,0.701640,7,0.445472,0.464817,0.0,0
586,0.438837,64.08,20.00,0.512223,0.512223,0.777971,0.602444,17.500,0.783849,0.701640,9,0.445472,0.464817,0.0,1000
140,0.472064,31.25,3.75,0.512223,0.512223,0.691912,0.602444,0.625,0.783849,0.701640,9,0.445472,0.464817,181.0,0
492,0.438837,39.25,9.50,0.512223,0.512223,0.424047,0.418882,6.500,0.783849,0.701640,14,0.452459,0.464817,240.0,4607
350,0.472064,26.17,2.00,0.512223,0.512223,0.272142,0.241546,0.000,0.072146,0.251652,0,0.445472,0.464817,276.0,1
